In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=2232d9a44c2c95555507d10b4b3d65e9d29316f5431573e4b3ee148c3eb4d298
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.7 GB  | Proc size: 159.0 MB
GPU RAM Free: 7611MB | Used: 0MB | Util   0% | Total 7611MB


In [ ]:
from google.colab import drive
drive.mount('/gdrive') 
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [ ]:
pip install mido

     |████████████████████████████████| 61kB 7.1MB/s 


In [ ]:
from mido import MidiFile, MidiTrack, Message
import numpy as np

num_notes = 118
samples_per_measure = 118

def midi_to_samples(fname):
    print(fname)
    has_time_sig = False
    flag_warning = False
    mid = MidiFile(fname)
    ticks_per_beat = mid.ticks_per_beat
    ticks_per_measure = 4 * ticks_per_beat

    for i, track in enumerate(mid.tracks):
        for msg in track:
            if msg.type == 'time_signature':
                new_tpm = msg.numerator * ticks_per_beat * 4 / msg.denominator
                if has_time_sig and new_tpm != ticks_per_measure:
                    flag_warning = True
                ticks_per_measure = new_tpm
                has_time_sig = True
    if flag_warning:
        print("  ^^^^^^ WARNING ^^^^^^")
        print("    " + fname)
        print("    Detected multiple distinct time signatures.")
        print("  ^^^^^^ WARNING ^^^^^^")
        return []

    all_notes = {}
    for i, track in enumerate(mid.tracks):
        abs_time = 0
        for msg in track:
            abs_time += msg.time
            if msg.type == 'note_on':
                if msg.velocity == 0:
                    continue
            # print("Msg : ",msg)
                note = msg.note - (128 - num_notes)/2
            # print(note)
                assert(note >= 0 and note < num_notes)
                if note not in all_notes:
                    all_notes[note] = []
                else:
                    single_note = all_notes[note][-1]
                    if len(single_note) == 1:
                        single_note.append(single_note[0] + 1)
                all_notes[note].append([abs_time * samples_per_measure / ticks_per_measure])
            elif msg.type == 'note_off':
                note = msg.note - (128 - num_notes)/2
                if note not in all_notes:
                    continue
                if len(all_notes[note][-1]) != 1:
                    continue
                all_notes[note][-1].append(abs_time * samples_per_measure / ticks_per_measure)
    for note in all_notes:
        for start_end in all_notes[note]:
            if len(start_end) == 1:
                start_end.append(start_end[0] + 1)
    samples = []
    for note in all_notes:
        for start, end in all_notes[note]:
            note=int(note)
      # print(note)
            sample_ix = int(start / samples_per_measure)
            while len(samples) <= sample_ix:
                samples.append(np.zeros((samples_per_measure, num_notes), dtype=np.uint8))
      # print(samples)
#             sample = samples[sample_ix]
      # print(sample_ix)
            start_ix = start - sample_ix * samples_per_measure
      # print(start_ix)
            if False:
                end_ix = min(end - sample_ix * samples_per_measure, samples_per_measure)
                while start_ix < end_ix:
                    samples[sample_ix][int(start_ix)][note] = 1
                    start_ix += 1
            else:
                samples[sample_ix][int(start_ix)][note] = 1
    return samples

def samples_to_midi(samples, fname, ticks_per_sample, thresh=0.5):
	mid = MidiFile()
	track = MidiTrack()
	mid.tracks.append(track)
	ticks_per_beat = mid.ticks_per_beat
	ticks_per_measure = 4 * ticks_per_beat
	ticks_per_sample = int(ticks_per_measure / samples_per_measure)
	abs_time = 0
	last_time = 0
	for sample in samples:
		for y in range(sample.shape[0]):
			abs_time += ticks_per_sample
			for x in range(sample.shape[1]):
				note = int(x + (128 - num_notes)/2)
				if sample[y,x] >= thresh and (y == 0 or sample[y-1,x] < thresh):
					delta_time = abs_time - last_time
					track.append(Message('note_on', note=note, velocity=127, time=delta_time))
					last_time = abs_time
				if sample[y,x] >= thresh and (y == sample.shape[0]-1 or sample[y+1,x] < thresh):
					delta_time = abs_time - last_time
					track.append(Message('note_off', note=note, velocity=127, time=delta_time))
					last_time = abs_time
	mid.save(fname)


In [ ]:
def transpose_range(samples):
	merged_sample = np.zeros_like(samples[0])
	for sample in samples:
		merged_sample = np.maximum(merged_sample, sample)
	merged_sample = np.amax(merged_sample, axis=0)
	min_note = np.argmax(merged_sample)
	max_note = merged_sample.shape[0] - np.argmax(merged_sample[::-1])
	return min_note, max_note

In [ ]:
def generate_add_centered_transpose(samples):
	num_notes = samples[0].shape[1]
	min_note, max_note = transpose_range(samples)
	s = int(num_notes/2 - (max_note + min_note)/2)
	out_samples = samples
	out_lens = [len(samples), len(samples)]
	for i in range(len(samples)):
		out_sample = np.zeros_like(samples[i])
		out_sample[:,min_note+s:max_note+s] = samples[i][:,min_note:max_note]
		out_samples.append(out_sample)
	return out_samples, out_lens

In [ ]:
import os
import numpy as np

patterns = {}
dirs = ["/gdrive/My Drive/midi_songs/", "download", "rag", "pop", "misc"]
all_samples = []
all_lens = []
print("Loading Songs...")
for dir in dirs:
    for root, subdirs, files in os.walk(dir):
        for file in files:
            path = root + '/' + file
            if not (path.endswith('.mid') or path.endswith('.midi')):
                continue
            # try:
            samples = midi_to_samples(path)
            # 			print(samples)
            # except:
            # 	print("ERROR ", path)
                # continue
            if len(samples) < 8:
                continue

            samples, lens = generate_add_centered_transpose(samples)
            all_samples += samples
            all_lens += lens

# print(all_samples)
assert(sum(all_lens) == len(all_samples))
print("Saving " + str(len(all_samples)) + " samples...")
all_samples = np.array(all_samples, dtype=np.uint8)
all_lens = np.array(all_lens, dtype=np.uint32)
np.save('/gdrive/My Drive/BeProject_v3/samples.npy', all_samples)
np.save('/gdrive/My Drive/BeProject_v3/lengths.npy', all_lens)
print("Done")

Loading Songs...
/gdrive/My Drive/midi_songs//0fithos.mid
  ^^^^^^ WARNING ^^^^^^
    /gdrive/My Drive/midi_songs//0fithos.mid
    Detected multiple distinct time signatures.
  ^^^^^^ WARNING ^^^^^^
/gdrive/My Drive/midi_songs//AT.mid
/gdrive/My Drive/midi_songs//ahead_on_our_way_piano.mid
/gdrive/My Drive/midi_songs//balamb.mid
/gdrive/My Drive/midi_songs//8.mid
/gdrive/My Drive/midi_songs//BlueStone_LastDungeon.mid
/gdrive/My Drive/midi_songs//bcm.mid
/gdrive/My Drive/midi_songs//caitsith.mid
/gdrive/My Drive/midi_songs//Cids.mid
/gdrive/My Drive/midi_songs//braska.mid
/gdrive/My Drive/midi_songs//cosmo.mid
/gdrive/My Drive/midi_songs//costadsol.mid
/gdrive/My Drive/midi_songs//dontbeafraid.mid
  ^^^^^^ WARNING ^^^^^^
    /gdrive/My Drive/midi_songs//dontbeafraid.mid
    Detected multiple distinct time signatures.
  ^^^^^^ WARNING ^^^^^^
/gdrive/My Drive/midi_songs//dayafter.mid
/gdrive/My Drive/midi_songs//DOS.mid
/gdrive/My Drive/midi_songs//decisive.mid
/gdrive/My Drive/midi_songs

In [ ]:
print((all_lens))

[ 75  75  84  84  70  70  21  21  78  78  60  60 104 104  64  64  34  34
 102 102  32  32  56  56  66  66  46  46  83  83  70  70  62  62  52  52
  30  30  68  68  89  89  39  39  48  48  92  92  24  24  24  24  57  57
  72  72  62  62 111 111  54  54  69  69  32  32  75  75  52  52  48  48
  27  27  35  35  41  41  68  68  34  34  50  50  81  81  94  94  88  88
  78  78  82  82  79  79 100 100  32  32  32  32  54  54  21  21  31  31
  94  94  40  40  28  28  59  59  35  35  74  74  66  66  60  60  43  43
  59  59  70  70  47  47  68  68  95  95  80  80  64  64  49  49  45  45
  34  34  64  64  30  30 122 122  43  43  48  48]


In [ ]:
# from mido import MidiFile, MidiTrack, Message, tick2second
# mid = MidiFile('/Users/drdevashishojaiswal/Documents/be_project/midi_songs/FFVII_BATTLE.mid')

In [ ]:
# print(mid.length)
# num_notes = 118
# samples_per_measure = 118
# has_time_sig=False
# flag_warning =False
# ticks_per_beat = mid.ticks_per_beat
# ticks_per_measure = 4 * ticks_per_beat
# for i, track in enumerate(mid.tracks):
#     for msg in track:
#         if msg.type == 'time_signature':
# #             print(msg)
#             new_tpm = msg.numerator * ticks_per_beat * 4 / msg.denominator
#             if has_time_sig and new_tpm != ticks_per_measure:
#                 flag_warning = True
#             ticks_per_measure = new_tpm
#             has_time_sig = True
# if flag_warning:
#     print("  ^^^^^^ WARNING ^^^^^^")
#     print("    " + fname)
#     print("    Detected multiple distinct time signatures.")
#     print("  ^^^^^^ WARNING ^^^^^^")
#     print("STOOOOOP")
# all_notes=dict()
# for i, track in enumerate(mid.tracks):
#     print(track)
#     abs_time=0
#     for msg in track:
# #         print(msg)
#         abs_time += msg.time
#         if msg.type == 'note_on':
#             if(msg.velocity == 0):
#                 continue
# #             print(msg.note,abs_time)
#             note = msg.note - (128 - num_notes)/2
# #             print(msg)
# #             if(msg.note<17):
# #                 print("Sdsdvvvvvsvadnacdhagcenacsdngcehcqhed")
#             assert(note >= 0 and note < num_notes)
#             if note not in all_notes:
#                 all_notes[note] = []
#             else:
#                 single_note = all_notes[note][-1]
#                 if len(single_note) == 1:
#                     single_note.append(single_note[0] + 1)
#             all_notes[note].append([abs_time * samples_per_measure / ticks_per_measure])
#         elif msg.type == 'note_off':
#             note = msg.note - (128 - num_notes)/2
#             if note not in all_notes:
#                 continue
#             if len(all_notes[note][-1]) != 1:
#                 continue
#             all_notes[note][-1].append(abs_time * samples_per_measure / ticks_per_measure)
# #         print(tick2second(abs_time,ticks_per_beat, 666666))
# # print(all_notes)
# for note in all_notes:
#         for start_end in all_notes[note]:
#             if len(start_end) == 1:
#                 start_end.append(start_end[0] + 1)
# # print(all_notes)
# samples = []
# for note in all_notes:
# #     print(note)
#     for start, end in all_notes[note]:
# #         print(start,end)
#         note=int(note)
#   # print(note)
#         sample_ix = int(start / samples_per_measure)
# #         print(sample_ix)
#         while len(samples) <= sample_ix:
#             samples.append(np.zeros((samples_per_measure, num_notes), dtype=np.uint8))
#   # print(samples)
# #         sample = samples[sample_ix]
#   # print(sample_ix)
#         start_ix = start - sample_ix * samples_per_measure
# #         print(sample_ix,start_ix)
#         if False:
#             end_ix = min(end - sample_ix * samples_per_measure, samples_per_measure)
#             while start_ix < end_ix:
#                 samples[sample_ix][int(start_ix)][note] = 1
#                 start_ix += 1
#         else:
#             samples[sample_ix][int(start_ix)][note] = 1
            
# print((samples[0][0]))


In [ ]:
# print(tick2second(abs_time))

In [ ]:
# if False:
#     print(dssdfsd)

In [ ]:
# print("Loading Data...")
# y_samples = np.load('/gdrive/My Drive/samples.npy')
# y_lengths = np.load('/gdrive/My Drive/lengths.npy')
# num_samples = y_samples.shape[0]
# num_songs = y_lengths.shape[0]
# print("Loaded " + str(num_samples) + " samples from " + str(num_songs) + " songs.")
# print(np.sum(y_lengths))
# assert(np.sum(y_lengths) == num_samples)

# print("Padding Songs...")
# x_shape = (num_songs * 1, 1)
# y_shape = (num_songs * 1, 16) + y_samples.shape[1:]
# x_orig = np.expand_dims(np.arange(x_shape[0]), axis=-1)
# y_orig = np.zeros(y_shape, dtype=y_samples.dtype)
# cur_ix = 0
# for i in range(num_songs):
# 	for ofs in range(1):
# 		ix = i*1 + ofs
# 		end_ix = cur_ix + y_lengths[i]
# 		for j in range(1):
# 			k = (j + ofs) % (end_ix - cur_ix)
# 			y_orig[ix,j] = y_samples[cur_ix + k]
# 	cur_ix = end_ix
# assert(end_ix == num_samples)
# x_train = np.copy(x_orig)
# y_train = np.copy(y_orig)


In [ ]:
# test_ix = 0
# y_test_song = np.copy(y_train[test_ix:test_ix+1])
# x_test_song = np.copy(x_train[test_ix:test_ix+1])
# samples_to_midi(y_test_song[0], '/Users/drdevashishojaiswal/Documents/be_project/gt.mid', 16)

NameError: ignored

In [ ]:
import sys, random, os
import numpy as np
from matplotlib import pyplot as plt

NUM_EPOCHS = 2000
LR = 0.001
CONTINUE_TRAIN = False
PLAY_ONLY = False
USE_EMBEDDING = False
USE_VAE = False
WRITE_HISTORY = True
NUM_RAND_SONGS = 10
DO_RATE = 0.1
BN_M = 0.9
VAE_B1 = 0.02
VAE_B2 = 0.1

BATCH_SIZE = 350
MAX_LENGTH = 16
PARAM_SIZE = 120
NUM_OFFSETS = 16 if USE_EMBEDDING else 1

def plotScores(scores, fname, on_top=True):
	plt.clf()
	ax = plt.gca()
	ax.yaxis.tick_right()
	ax.yaxis.set_ticks_position('both')
	ax.yaxis.grid(True)
	plt.plot(scores)
	plt.ylim([0.0, 0.009])
	plt.xlabel('Epoch')
	loc = ('upper right' if on_top else 'lower right')
	plt.draw()
	plt.savefig(fname)

def save_config():
	with open('/gdrive/My Drive/BeProject_v3/config.txt', 'w') as fout:
		fout.write('LR:          ' + str(LR) + '\n')
		fout.write('BN_M:        ' + str(BN_M) + '\n')
		fout.write('BATCH_SIZE:  ' + str(BATCH_SIZE) + '\n')
		fout.write('NUM_OFFSETS: ' + str(NUM_OFFSETS) + '\n')
		fout.write('DO_RATE:     ' + str(DO_RATE) + '\n')
		fout.write('num_songs:   ' + str(num_songs) + '\n')
		fout.write('optimizer:   ' + type(model.optimizer).__name__ + '\n')

###################################
#  Load Keras
###################################
print("Loading Keras...")
import os, math
# os.environ['THEANORC'] = "./gpu.theanorc"
# os.environ['KERAS_BACKEND'] = "theano"
# import theano
# print("Theano Version: " + theano.__version__)

import keras
print("Keras Version: " + keras.__version__)
from keras.layers import Input, Dense, Activation, Dropout, Flatten, Reshape, Permute, RepeatVector, ActivityRegularization, TimeDistributed, Lambda, SpatialDropout1D
from keras.layers.convolutional import Conv1D, Conv2D, Conv2DTranspose, UpSampling2D, ZeroPadding2D
from keras.layers.embeddings import Embedding
from keras.layers.local import LocallyConnected2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.noise import GaussianNoise
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.initializers import RandomNormal
from keras.losses import binary_crossentropy
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.utils import plot_model
from keras import backend as K
from keras import regularizers
from keras.engine.topology import Layer
K.set_image_data_format('channels_first')

#Fix the random seed so that training comparisons are easier to make
# np.random.seed(0)
# random.seed(0)

if WRITE_HISTORY:
	#Create folder to save models into
	if not os.path.exists('/gdrive/My Drive/BeProject_v3/History'):
		os.makedirs('/gdrive/My Drive/BeProject_v3/History')

###################################
#  Load Dataset
###################################
print("Loading Data...")
y_samples = np.load('/gdrive/My Drive/samples.npy')
y_lengths = np.load('/gdrive/My Drive/lengths.npy')
num_samples = y_samples.shape[0]
num_songs = y_lengths.shape[0]
print("Loaded " + str(num_samples) + " samples from " + str(num_songs) + " songs.")
print(np.sum(y_lengths))
assert(np.sum(y_lengths) == num_samples)

print("Padding Songs...")
x_shape = (num_songs * NUM_OFFSETS, 1)
y_shape = (num_songs * NUM_OFFSETS, MAX_LENGTH) + y_samples.shape[1:]
x_orig = np.expand_dims(np.arange(x_shape[0]), axis=-1)
y_orig = np.zeros(y_shape, dtype=y_samples.dtype)
cur_ix = 0
for i in range(num_songs):
	for ofs in range(NUM_OFFSETS):
		ix = i*NUM_OFFSETS + ofs
		end_ix = cur_ix + y_lengths[i]
		for j in range(MAX_LENGTH):
			k = (j + ofs) % (end_ix - cur_ix)
			y_orig[ix,j] = y_samples[cur_ix + k]
	cur_ix = end_ix
assert(end_ix == num_samples)
x_train = np.copy(x_orig)
y_train = np.copy(y_orig)

def to_song(encoded_output):
	return np.squeeze(decoder([np.round(encoded_output), 0])[0])

def reg_mean_std(x):
	s = K.log(K.sum(x * x))
	return s*s

def vae_sampling(args):
	z_mean, z_log_sigma_sq = args
	epsilon = K.random_normal(shape=K.shape(z_mean), mean=0.0, stddev=VAE_B1)
	return z_mean + K.exp(z_log_sigma_sq * 0.5) * epsilon

def vae_loss(x, x_decoded_mean):
	xent_loss = binary_crossentropy(x, x_decoded_mean)
	kl_loss = VAE_B2 * K.mean(1 + z_log_sigma_sq - K.square(z_mean) - K.exp(z_log_sigma_sq), axis=None)
	return xent_loss - kl_loss
	
test_ix = 0
y_test_song = np.copy(y_train[test_ix:test_ix+1])
x_test_song = np.copy(x_train[test_ix:test_ix+1])
samples_to_midi(y_test_song[0], '/gdrive/My Drive/BeProject_v3/gt.mid', 16)

###################################
#  Create Model
###################################
if CONTINUE_TRAIN or PLAY_ONLY:
	print("Loading Model...")
	model = load_model('model.h5', custom_objects=custom_objects)
else:
	print("Building Model...")

	if USE_EMBEDDING:
		x_in = Input(shape=x_shape[1:])
		print((None,) + x_shape[1:])
		x = Embedding(x_train.shape[0], PARAM_SIZE, input_length=1)(x_in)
		x = Flatten(name='pre_encoder')(x)
	else:
		x_in = Input(shape=y_shape[1:])
		print((None,) + y_shape[1:])
		x = Reshape((y_shape[1], -1))(x_in)
		print(K.int_shape(x))
		
		x = TimeDistributed(Dense(2000, activation='relu'))(x)
		print(K.int_shape(x))
		
		x = TimeDistributed(Dense(200, activation='relu'))(x)
		print(K.int_shape(x))

		x = Flatten()(x)
		print(K.int_shape(x))

		x = Dense(1600, activation='relu')(x)
		print(K.int_shape(x))
		
		if USE_VAE:
			z_mean = Dense(PARAM_SIZE)(x)
			z_log_sigma_sq = Dense(PARAM_SIZE)(x)
			x = Lambda(vae_sampling, output_shape=(PARAM_SIZE,), name='pre_encoder')([z_mean, z_log_sigma_sq])
		else:
			x = Dense(PARAM_SIZE)(x)
			x = BatchNormalization(momentum=BN_M, name='pre_encoder')(x)
	print(K.int_shape(x))
	
	x = Dense(1600, name='encoder')(x)
	x = BatchNormalization(momentum=BN_M)(x)
	x = Activation('relu')(x)
	if DO_RATE > 0:
		x = Dropout(DO_RATE)(x)
	print(K.int_shape(x))

	x = Dense(MAX_LENGTH * 200)(x)
	print(K.int_shape(x))
	x = Reshape((MAX_LENGTH, 200))(x)
	x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
	x = Activation('relu')(x)
	if DO_RATE > 0:
		x = Dropout(DO_RATE)(x)
	print(K.int_shape(x))

	x = TimeDistributed(Dense(2000))(x)
	x = TimeDistributed(BatchNormalization(momentum=BN_M))(x)
	x = Activation('relu')(x)
	if DO_RATE > 0:
		x = Dropout(DO_RATE)(x)
	print(K.int_shape(x))

	x = TimeDistributed(Dense(y_shape[2] * y_shape[3], activation='sigmoid'))(x)
	print(K.int_shape(x))
	x = Reshape((y_shape[1], y_shape[2], y_shape[3]))(x)
	print(K.int_shape(x))
	
	if USE_VAE:
		model = Model(x_in, x)
		model.compile(optimizer=Adam(lr=LR), loss=vae_loss)
	else:
		model = Model(x_in, x)
		model.compile(optimizer=RMSprop(lr=LR), loss='binary_crossentropy')

	plot_model(model, to_file='/gdrive/My Drive/BeProject_v3/model.png', show_shapes=True)

###################################
#  Train
###################################
print("Compiling SubModels...")
func = K.function([model.get_layer('encoder').input, K.learning_phase()],
				  [model.layers[-1].output])
enc = Model(inputs=model.input, outputs=model.get_layer('pre_encoder').output)

rand_vecs = np.random.normal(0.0, 1.0, (NUM_RAND_SONGS, PARAM_SIZE))
np.save('/gdrive/My Drive/BeProject_v3/rand.npy', rand_vecs)

def make_rand_songs(write_dir, rand_vecs):
	for i in range(rand_vecs.shape[0]):
		x_rand = rand_vecs[i:i+1]
		y_song = func([x_rand, 0])[0]
		samples_to_midi(y_song[0], write_dir + 'rand' + str(i) + '.mid', 16, 0.25)

def make_rand_songs_normalized(write_dir, rand_vecs):
	if USE_EMBEDDING:
		x_enc = np.squeeze(enc.predict(x_orig))
	else:
		x_enc = np.squeeze(enc.predict(y_orig))
	
	x_mean = np.mean(x_enc, axis=0)
	x_stds = np.std(x_enc, axis=0)
	x_cov = np.cov((x_enc - x_mean).T)
	u, s, v = np.linalg.svd(x_cov)
	e = np.sqrt(s)

	print("Means: ", x_mean[:6])
	print("Evals: ", e[:6])
	
	np.save(write_dir + 'means.npy', x_mean)
	np.save(write_dir + 'stds.npy', x_stds)
	np.save(write_dir + 'evals.npy', e)
	np.save(write_dir + 'evecs.npy', v)

	x_vecs = x_mean + np.dot(rand_vecs * e, v)
	make_rand_songs(write_dir, x_vecs)
	
	title = ''
	if '/' in write_dir:
		title = 'Epoch: ' + write_dir.split('/')[-2][1:]
	
	plt.clf()
	e[::-1].sort()
	plt.title(title)
	plt.bar(np.arange(e.shape[0]), e, align='center')
	plt.draw()
	plt.savefig(write_dir + 'evals.png')

	plt.clf()
	plt.title(title)
	plt.bar(np.arange(e.shape[0]), x_mean, align='center')
	plt.draw()
	plt.savefig(write_dir + 'means.png')
	
	plt.clf()
	plt.title(title)
	plt.bar(np.arange(e.shape[0]), x_stds, align='center')
	plt.draw()
	plt.savefig(write_dir + 'stds.png')

if PLAY_ONLY:
	print("Generating Songs...")
	make_rand_songs_normalized('', rand_vecs)
	for i in range(20):
		x_test_song = x_train[i:i+1]
		y_song = model.predict(x_test_song, batch_size=BATCH_SIZE)[0]
		samples_to_midi(y_song, '/gdrive/My Drive/BeProject_v3/gt' + str(i) + '.mid', 16)
	exit(0)
		  
print("Training...")
save_config()
train_loss = []
ofs = 0

for iter in range(NUM_EPOCHS):
	if USE_EMBEDDING:
		history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=1)
	else:
		cur_ix = 0
		for i in range(num_songs):
			end_ix = cur_ix + y_lengths[i]
			for j in range(MAX_LENGTH):
				k = (j + ofs) % (end_ix - cur_ix)
				y_train[i,j] = y_samples[cur_ix + k]
			cur_ix = end_ix
		assert(end_ix == num_samples)
		ofs += 1

		history = model.fit(y_train, y_train, batch_size=BATCH_SIZE, epochs=1)

	loss = history.history["loss"][-1]
	train_loss.append(loss)
	print("Train Loss: " + str(train_loss[-1]))
	
	if WRITE_HISTORY:
		plotScores(train_loss, '/gdrive/My Drive/BeProject_v3/History/Scores.png', True)
	else:
		plotScores(train_loss, '/gdrive/My Drive/BeProject_v3/Scores.png', True)
	
	i = iter + 1
	if i in [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 200, 250, 300, 350, 400, 450] or (i % 100 == 0):
		write_dir = ''
		if WRITE_HISTORY:
			#Create folder to save models into
			write_dir = '/gdrive/My Drive/BeProject_v3/History/e' + str(i)
			if not os.path.exists(write_dir):
				os.makedirs(write_dir)
			write_dir += '/'
			model.save('/gdrive/My Drive/BeProject_v3/History/model.h5')
		else:
			model.save('/gdrive/My Drive/BeProject_v3/model.h5')
		print("Saved")

		if USE_EMBEDDING:
			y_song = model.predict(x_test_song, batch_size=BATCH_SIZE)[0]
		else:
			y_song = model.predict(y_test_song, batch_size=BATCH_SIZE)[0]
		# util.samples_to_pics(write_dir + 'test', y_song)
	  samples_to_midi(y_song, write_dir + 'test.mid', 16)

		make_rand_songs_normalized(write_dir, rand_vecs)

print("Done")
